In [1]:
# Install and import packages into mlenv enviroment 
from sentence_transformers import SentenceTransformer, util
import pandas as pd

In [2]:
# Define model from sentence_transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
# Import cleaned data csvs
file_anime = 'Final Resources/cleaned_anime.csv'
file_la = 'Final Resources/cleaned_live_actions.csv'

In [4]:
# Create dfs
anime_df = pd.read_csv(file_anime, index_col=0) # add index_col=0 so that there aren't 2 index columns!
la_df = pd.read_csv(file_la, index_col=0)

In [5]:
# Make a list of anime descriptions from df
anime_desc = anime_df['synopsis'].tolist()

In [6]:
# Make a list of netflix, hulu, amazon prime, disney+ descriptions
la_desc = la_df['description'].tolist()

In [7]:
# Encode anime descriptions 
anime_embedding = model.encode(anime_desc) # Encoding the anime list

In [8]:
# Encode live action descriptions 
la_embedding = model.encode(la_desc) # Encoding the netflix list

In [9]:
anime_df["title"][1]

'Cowboy Bebop'

In [10]:
# Compute cosine similarities
cos_sim = util.cos_sim(la_embedding, anime_embedding)
cos_sim

tensor([[ 0.0485,  0.2290,  0.2079,  ...,  0.1509,  0.1470,  0.0789],
        [ 0.1681,  0.1254,  0.1798,  ...,  0.2372,  0.0299,  0.1367],
        [ 0.2328,  0.1589,  0.2318,  ...,  0.1661,  0.2453,  0.0818],
        ...,
        [ 0.0841, -0.0547,  0.2750,  ...,  0.0641,  0.1564, -0.0754],
        [ 0.1459,  0.1325,  0.0592,  ...,  0.0531,  0.1309,  0.0949],
        [-0.0379,  0.1005, -0.0381,  ...,  0.0326,  0.1212,  0.0663]])

In [11]:
# Build reccomendations column in la_df from cos_sim tensor

recs = []
# for each row in the cos_sim tensor
for i in range(len(cos_sim)):
    mydict={}

    # for each col in the row
    for x in range(len(cos_sim[i])):
        #assign title to the column from anime_df["title"][col#]
        try:
            mydict[anime_df["title"][x]] = cos_sim[i][x]
        except KeyError:
            continue

    # find the max value in all the columns
    top2 = sorted(mydict, key=mydict.get, reverse=True)[:5]
    recs.append(top2)

la_df['recommended_animes'] = recs

la_df.head()

,title,description,release_year,type,listed_in,source,recommended_animes
show_id,,,,,,,
n-s1,Dick Johnson Is Dead,"As her father nears the end of his life, filmm...",2020,Movie,Documentaries,Netflix,"[s.CRY.ed, Mobile Suit Gundam SEED C.E.73: Sta..."
n-s2,Blood & Water,"After crossing paths at a party, a Cape Town t...",2021,TV Show,"International TV Shows, TV Dramas, TV Mysteries",Netflix,"[Sky Girls, Arashi no Yoru ni, Digimon Savers,..."
n-s3,Ganglands,To protect his family from a powerful drug lor...,2021,TV Show,"Crime TV Shows, International TV Shows, TV Act...",Netflix,"[Bleach, Aa! Megami-sama!: Sorezore no Tsubasa..."
n-s4,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo...",2021,TV Show,"Docuseries, Reality TV",Netflix,"[Minky Momo in Yume ni Kakeru Hashi, Kimi ga A..."
n-s5,Kota Factory,In a city of coaching centers known to train I...,2021,TV Show,"International TV Shows, Romantic TV Shows, TV ...",Netflix,"[Doraemon Movie 23: Nobita to Robot Kingdom, O..."


In [22]:
la_df_export.reset_index()

,show_id,title,recommended_animes
0,n-s1,Dick Johnson Is Dead,"[s.CRY.ed, Mobile Suit Gundam SEED C.E.73: Sta..."
1,n-s2,Blood & Water,"[Sky Girls, Arashi no Yoru ni, Digimon Savers,..."
2,n-s3,Ganglands,"[Bleach, Aa! Megami-sama!: Sorezore no Tsubasa..."
3,n-s4,Jailbirds New Orleans,"[Minky Momo in Yume ni Kakeru Hashi, Kimi ga A..."
4,n-s5,Kota Factory,"[Doraemon Movie 23: Nobita to Robot Kingdom, O..."
...,...,...,...
22959,a-s9664,Pride Of The Bowery,"[Sousei no Aquarion, xxxHOLiC, Doraemon: Nobit..."
22960,a-s9665,Planet Patrol,"[Bokura no, PopoloCrois, Muumindani no Suisei,..."
22961,a-s9666,Outpost,"[Ryuusei no Rockman, Keroro Gunsou Movie 1, Er..."
22962,a-s9667,Maradona: Blessed Dream,"[Giant Gorg, Captain Tsubasa, Princess Nine: K..."


In [14]:
# make new df with only the columns we want
la_df_export = la_df[['title', 'recommended_animes']]
la_df_export

,title,recommended_animes
show_id,,
n-s1,Dick Johnson Is Dead,"[s.CRY.ed, Mobile Suit Gundam SEED C.E.73: Sta..."
n-s2,Blood & Water,"[Sky Girls, Arashi no Yoru ni, Digimon Savers,..."
n-s3,Ganglands,"[Bleach, Aa! Megami-sama!: Sorezore no Tsubasa..."
n-s4,Jailbirds New Orleans,"[Minky Momo in Yume ni Kakeru Hashi, Kimi ga A..."
n-s5,Kota Factory,"[Doraemon Movie 23: Nobita to Robot Kingdom, O..."
...,...,...
a-s9664,Pride Of The Bowery,"[Sousei no Aquarion, xxxHOLiC, Doraemon: Nobit..."
a-s9665,Planet Patrol,"[Bokura no, PopoloCrois, Muumindani no Suisei,..."
a-s9666,Outpost,"[Ryuusei no Rockman, Keroro Gunsou Movie 1, Er..."


In [23]:
# Export new json
la_df_export.to_json('Updated Website/live_actions_with_anime_recs.json', orient='records',lines=True)

In [21]:
# Export new json
la_df_export.to_csv('Updated Website/live_actions_with_anime_recs.csv', index=True)

['{"title":"Dick Johnson Is Dead","recommended_animes":["s.CRY.ed","Mobile Suit Gundam SEED C.E.73: Stargazer","HeatGuy J Special","Aria The Animation","Rockman.EXE Beast"]}',
 '{"title":"Blood & Water","recommended_animes":["Sky Girls","Arashi no Yoru ni","Digimon Savers","Hachimitsu to Clover Specials","Bamboo Blade"]}',
 '{"title":"Ganglands","recommended_animes":["Bleach","Aa! Megami-sama!: Sorezore no Tsubasa Specials","Lodoss-tou Senki","Akachan to Boku","Detective Conan Movie 10: Requiem of the Detectives"]}',
 '{"title":"Jailbirds New Orleans","recommended_animes":["Minky Momo in Yume ni Kakeru Hashi","Kimi ga Aruji de Shitsuji ga Ore de","Rurouni Kenshin: Meiji Kenkaku Romantan - Ishinshishi e no Chinkonka","Code Geass: Hangyaku no Lelouch Recaps","Uchuu Kaizoku Captain Herlock"]}',
 '{"title":"Kota Factory","recommended_animes":["Doraemon Movie 23: Nobita to Robot Kingdom","Ojamajo Doremi Sharp","Taiho Shichau zo Special","Loveless","Shuffle!"]}',
 '{"title":"Midnight Mass","